<a href="https://colab.research.google.com/github/Lakshita2002/FER_brain_and_cognitive_society/blob/master/Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline
import cv2
import os
import math
from mlxtend.image import extract_face_landmarks
import gc
#Essential Keras Functions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input,Dense,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.backend import clear_session
#Essential sklearn Functions
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

In [ ]:
#LAKSHITA

# loading images in the form of an array
jaffe_dir_path = '/content/drive/My Drive/Images/'
expres_code = ['NE','HA','AN','DI','FE','SA','SU']
expressions = [ 0,   1,   2,   3,   4,   5,   6]
img_names = []
img_data_list = []
labels = []

img_list = os.listdir(jaffe_dir_path)
for img in img_list:
    input_img = cv2.imread(jaffe_dir_path + img, cv2.IMREAD_GRAYSCALE)
    img_data_list.append(input_img)
    
img_data = np.array(img_data_list)

print(img_data.shape)

In [ ]:
def rotate_image(image,angle):
    '''
    Returns a rotated image given the angle(degrees) to be rotated and image
    '''
    rows,cols = image.shape
    #Transformation Matrix(M)
    M = cv2.getRotationMatrix2D(((rows - 1)/2.0,(cols - 1)/2.0),angle,1)
    rot_img = cv2.warpAffine(image, M, (cols,rows))

    return rot_img

In [ ]:
def find_angle(point1, point2):
    '''
    To find angle in degrees for the given two points
    '''
    # angle in radians
    angle_r = math.atan((point2[1] - point1[1])/(point2[0] - point1[0]))
    # angle in degrees
    angle_d = math.degrees(angle_r)

    return angle_d

In [ ]:
def eye_centers(landmarks):
    '''
    To find the eye centers(mean of the 6 landmark points around the eye)
    '''
    #36-41 are landmark points surrounding right eye
    point1 = (np.mean(landmarks[36:42,0]),np.mean(landmarks[36:42,1]))
    #42-47 are landmark points surrounding left eye
    point2 = (np.mean(landmarks[42:48,0]),np.mean(landmarks[42:48,1]))

    return point1, point2

In [ ]:
def preprocessing(input_images):

    '''
    The preprocessing involves rotation of the image and image cropping
    Arguments :
    input_images -- array of images of shape (m,h,w)
    Returns :
    array of images after preprocessing
    '''
    preprocessed_faces = []
    for img in input_images:
        
        #landmark detection
        #(returns an array of landmarks of shape (68,2))
        landmarks = extract_face_landmarks(img)

        #detect eye cnters
        p1, p2 = eye_centers(landmarks)

        #find angle 
        angle = find_angle(p1, p2)
        
        #rotate image
        rot_img = rotate_image(img, angle)

        #find length 'd'
        p1_new, p2_new = eye_centers(extract_face_landmarks(rot_img))
        d = cv2.norm(np.array(p1_new) - np.array(p2_new))

        #mid point of new eye centers
        d_mid = ((p2_new[0] + p1_new[0])/2.0 , (p2_new[1] + p1_new[1])/2.0)

        #point above line joining eye centers
        x_up = d_mid[0]
        y_up = d_mid[1] - (0.6*d)

        #cropping image
        x_start = int(landmarks[0,0])
        x_end = int(landmarks[16,0])
        y_start = int(y_up)
        y_end = int(landmarks[8,1])

        crop_img = img[y_start:y_end,x_start:x_end]

        #resize the cropped image
        face_roi = cv2.resize(crop_img,(48,48))
        
        preprocessed_faces.append(face_roi)

    return np.array(preprocessed_faces)

In [ ]:
#PRANAY
def normalization(imagedata, mean, std_dev):
    '''
    To apply Histogram equalization and Z-Square Normalization to the preprocessed images
    Arguments :
    imagedata -- array of preprocessed images of shape (m,h,w)
    mean -- mean of imagedata array
    std_dev -- standard deviation of imagedata array
    Returns :
    array of normalized images of shape (m,48,48)
    '''
    normalized_images = []
    for i in range(imagedata.shape[0]):
        #Histogram Equalization
        hist_eqv = cv2.equalizeHist(imagedata[i])

        #Z-Square normalization
        zsq_norm = ((hist_eqv - mean)/std_dev)

        #Resize
        resized_image = cv2.resize(zsq_norm, (48,48))
        normalized_images.append(resized_image)

    return np.array(normalized_images)

In [ ]:
#KUSUM
def data_aug(X_train,X_val,y_train,y_val,train_batch_size,val_batch_size):
    '''
    To apply data augmentation on training data.
    Arguments :
    X_train and X_val -- array of training and validation data containing images
    y_train and y_val -- labels of training and validation images
    train_batch_size and val_batch_size contains the batch size of images
    Returns:
    Iterator for training and validation batch 
    '''
    #create image data augmenatation generator for training and validation data
    train_datagen = ImageDataGenerator(rotation_range=2,
                                       rescale=1.0,
                                       horizontal_flip=True,
                                       fill_mode='nearest')
    
    val_datagen = ImageDataGenerator(rescale=1.0)

    #create iterator for training and validation data
    train_batch = train_datagen.flow(X_train,y_train,batch_size=train_batch_size)


    val_batch = val_datagen.flow(X_val,y_val,batch_size=val_batch_size)


    return (train_batch,val_batch)

In [ ]:
#PRANAY
def augmentation(imagedata):
    '''
    To increase the no. of images by Random rotation and Horizontal flipping
    and all the images genarated are stored in the specified directory with proper name
    Arguments :
    imagedata -- array of images after preprocessing and normalization
    '''

    for i in range(imagedata.shape[0]):
        img = imagedata[i].reshape(imagedata[i].shape + (1,))
        img = array_to_img(img)#To convert to PIL image instance
        img = img_to_array(img)
        img = np.array([img])
        
        datagen = ImageDataGenerator(rotation_range = 3,horizontal_flip = True)

        imgGen = datagen.flow(img, batch_size = 1, save_to_dir =
                              'drive/My Drive/Augmented Images/jaffeaug',
                              save_prefix = imagename[i], save_format='tiff')

      for j in range(5): #generates 5 images per image
        
          batch = imgGen.next()

In [ ]:
#KUSUM
def datareshaping(normalized_data,labels):
    '''
    Expands the dimensions of the normalized_data and and one-hot encodes the target labels
    Arguments:
    normalized_data -- normalized_data obtained after preforming normalization and downsampling of shape (m,48,48)
    labels -- contains the 1-dimensional array of labels
    Returns:
    expanded normalized data of shape(m,48,48,1) and one hot encoded target labels of shape(m,6)

    '''
    X = normalized_data
    X = np.reshape(X ,(normalized_data.shape + (1,))

    y = to_categorical(labels)
    return X,y

In [ ]:
def recog_model(input_shape):
    '''
    Create the keras model architecture
    Arguments:
    input_shape -- The dimensions of the input data
    Returns:
    The created model
    '''

    # input placeholder as a tensor of input_shape
    X_input = Input(input_shape)

    # define the keras model
    X = Conv2D(48, (5,5), strides = (1,1), padding = 'valid', name = 'conv1')(X_input)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)

    X = MaxPooling2D((2,2), strides = (2,2), padding = 'valid', name = 'maxpool1')(X)

    X = Conv2D(64, (5,5), strides = (1,1), padding = 'valid', activation = 'relu', name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)

    X = MaxPooling2D((2,2), strides = (2,2), padding = 'valid', name = 'maxpool2')(X)

    X = Flatten()(X)
    X = Dropout(0.5)(X)
    X = Dense(units = 7,activation = 'softmax',name = 'out',kernel_initializer = 'glorot_normal')(X)

    model = Model(inputs = X_input, outputs = X, name = 'recog_model')
    sgd = SGD(learning_rate = 0.001, momentum = 0.9)

    #compile the keras model
    model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

    return model

In [ ]:
def traintestsplit(X,y):
    '''
    Splits the train and test data into 70:30 ratio
    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return X_train,X_test,y_train,y_test

In [ ]:
#LAKSHITA

# landmark identification
for i in range (0,img_data[:,0,0].size ):
	
	faces_in_image = detector(img_data[i], 0)

	for face in faces_in_image:

		landmarks = predictor(img_data[i], face)

		landmarks_list = []
		for j in range(0, landmarks.num_parts):
			landmarks_list.append((landmarks.part(j).x, landmarks.part(j).y))

		for landmark_num, xy in enumerate(landmarks_list, start = 1):
			cv2.circle(img_data[i], (xy[0], xy[1]), 1, (168, 0, 20), -1)

cv2_imshow(img_data[157])

In [ ]:
#LAKSHITA

# image cropping
cropped_images = preprocessing(img_data)
cv2_imshow(cropped_images[1])

In [ ]:
#Pranay,#Kusum
def K_fold_cv(X_tmp, Y_tmp,n_splits = 10, batch_size = 16, epochs = 120):
    '''
    Applies K fold cross validation for given Training data
    Returns the Mean and standard deviation of accuracies
    '''
  
    i = 1
    model_accuracies = []
    for train_index, val_index in KFold(n_splits = n_splits, shuffle = True, random_state = 0).split(X_tmp, Y_tmp):
      
        X_train, X_val = X_tmp[train_index], X_tmp[val_index]
        X_train = X_train.astype('float32')
        X_val = X_val.astype('float32')
        Y_train, Y_val = Y_tmp[train_index], Y_tmp[val_index]
        
        print('Model evaluation : ' + str(i))
        
        #create a model object
        Recog_Model = recog_model(X_tmp.shape[1:])
        #train the model
        Recog_Model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs,
                        shuffle = True, validation_data = (X_val, Y_val))
        
        #plotting validation loss and training loss vs epochs
        loss_train = history.history['loss']
        loss_val = history.history['val_loss']
        epochs = range(epochs)
        plt.plot(epochs, loss_train, 'g', label = 'training_loss')
        plt.plot(epochs, loss_val, 'b', label = 'val_loss')
        plt.title('Loss Vs Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f'drive/My Drive/ER project/Jaffe(foldf{i})losses.png' )
        plt.clf()

        #plotting validation accuracy and training accuracy vs epochs
        accuracy_train = history.history['accuracy']
        accuracy_val = history.history['val_accuracy']
        plt.plot(epochs, accuracy_train, 'g', label = 'training_accuracy')
        plt.plot(epochs, accuracy_val, 'b', label = 'val_accuracy')
        plt.title('Accuracy Vs Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.savefig(f'drive/My Drive/ER project/Jaffe(foldf{i})accuracies.png')
        plt.clf()

        metrics = Recog_Model.evaluate(X_val, Y_val)
        print(f'Accuracy fold f{}: ' + str(metrics[1] * 100))
        model_accuracies.append(metrics[1] * 100)
        Recog_Model.save(f'drive/My Drive/ER project/Jaffe(foldf{i}).h5')
        #Discard the present model
        del Recog_Model

        gc.collect()
        clear_session()
        i += 1

    #calculating mean and standard deviation of accuracies
    Mean_Accuracy = np.mean(model_accuracies)
    standard_deviation = np.std(model_accuracies)

    return Mean_Accuracy,standard_deviation